- <font style ="color:purple;font-size:20px;">Gurobipy for resource allocation </font>

- <font color = "black" >Two factories <font color = "purple">$ F1 $ , $ F2 $ </font> with supplies <font color = "purple">100 , 200 </font>units resp.</font>
- <font color = "black" >Three shops <font color = "purple">$ S1 $ , $ S2 $ , $S3$ </font> with demands <font color = "purple">50 , 100 , 150 </font>units resp.</font>
- <font color = "black" >Shipping cost per unit:</font>

<table style="width:50%" >
  <tr>
    <th style="color: purple ; font-family:verdana ;text-align:center;"></th>
    <th style="color: purple ; font-family:verdana ;text-align:center;">$S1$</th>
    <th style="color: purple ; font-family:verdana ;text-align:center;">$S2$</th> 
    <th style="color: purple ; font-family:verdana ;text-align:center;">$S3$</th>
  </tr>
     <tr>
    <td style="color: purple ; font-family:verdana ;text-align:center;">$F1$</td>
    <td style="color: purple ; font-family:verdana ;text-align:center;">10</td>
    <td style="color: purple ; font-family:verdana ;text-align:center;">12</td> 
    <td style="color: purple ; font-family:verdana ;text-align:center;">14</td>
  </tr>
     <tr>
    <td style="color: purple ; font-family:verdana ;text-align:center;">$F2$</td>
    <td style="color: purple ; font-family:verdana ;text-align:center;">11</td>
    <td style="color: purple ; font-family:verdana ;text-align:center;">12</td> 
    <td style="color: purple ; font-family:verdana ;text-align:center;">13</td>
  </tr>
    


- $x_{ij}$- Number of units to ship from factory $i$ to shop $j$
- objective : minimize shipping cost
$$\begin{align}min \quad 10x_{11} + 12x_{12} + 14x_{13} + 11x_{21} + 12x_{22} + 13x_{23} \end{align}$$
- Supply constraints:
$$\begin{align}
F1 : x_{11} + x_{12} + x_{13} \leq 100 \\
F2 : x_{21} + x_{22} + x_{23} \leq 200 \end{align}$$
- Demand constraints:
$$\begin{align}
S1 : x_{11} + x_{21}  \geq 50 \\
S2 : x_{12} + x_{22}  \geq 100 \\
S3 : x_{13} + x_{23}  \geq 150 \\
\end{align}$$
- Nonnegetivivty constraints:can not ship negetive amount of units
$$\begin{align}x_{11} + x_{12} + x_{13} + x_{21} + x_{22} + x_{23} \geq 0 \end{align}$$

In [1]:
from gurobipy import *
import numpy as np

In [2]:
F = [1, 2]
S = [1, 2 , 3]

In [3]:
# comb is x11 ,x12 ,x13 , ... 
# ms is the multipliers of x11--->(F1 , S1)=10
comb , ms = multidict({(1 , 1):10,
                      (1 ,  2):12,
                      (1, 3):14,
                      (2, 1):11,
                      (2 , 2):12,
                      (2, 3):13})

constr = [100, 200, -50, -100, -150]

In [4]:
m = Model("Transportation")

Restricted license - for non-production use only - expires 2022-01-13


In [5]:
x = m.addVars(comb)
m.update()
x

{(1, 1): <gurobi.Var C0>,
 (1, 2): <gurobi.Var C1>,
 (1, 3): <gurobi.Var C2>,
 (2, 1): <gurobi.Var C3>,
 (2, 2): <gurobi.Var C4>,
 (2, 3): <gurobi.Var C5>}

In [6]:
m.addConstrs(x.sum(i,"*")<=100 for i in range(1,2) )  
m.addConstrs(x.sum(j,"*")<=200 for j in range(2,3) ) 


{2: <gurobi.Constr *Awaiting Model Update*>}

In [7]:
m.addConstrs(x.sum("*" , s)>=50 for s in range(1,2) )
m.addConstrs(x.sum("*" , p)>=100 for p in range(2,3) )
m.addConstrs(x.sum("*" , o)>=150 for o in range(3,4) )
m.addConstrs(x.sum("*" , f)>=0 for f in range(1,4) )

{1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [8]:
m.modelSense = GRB.MINIMIZE

In [9]:
m.setObjective(x.prod(ms))

In [10]:
m.write("m.lp")

In [11]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 8 rows, 6 columns and 18 nonzeros
Model fingerprint: 0x75669f87
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 2e+02]
Presolve removed 3 rows and 0 columns
Presolve time: 0.02s
Presolved: 5 rows, 6 columns, 12 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9500000e+03   1.875000e+01   0.000000e+00      0s
       3    3.6500000e+03   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds
Optimal objective  3.650000000e+03


In [12]:
for v in m.getVars():
	print('%s %g' % (v.varName, v.x))
print('Obj: %g' % m.objVal)

C0 50
C1 50
C2 0
C3 0
C4 50
C5 150
Obj: 3650
